In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from bs4 import BeautifulSoup
import re
from random import randint
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pickle
import requests
import math
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import cfscrape
from lxml import etree
from datetime import date

#### Initiate webdriver

In [ ]:
browser = webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver')

### Getting products links

In [ ]:
products_collection = ['https://www.scs-holzshop.de/holzbalken/', 
                      'https://www.scs-holzshop.de/hobelware-latten/',
                      'https://www.scs-holzshop.de/platten-daemmstoffe/',
                      'https://www.scs-holzshop.de/terrasse/',
                      'https://www.scs-holzshop.de/holzbalken/']

In [ ]:
%%time
all_links = []
for n in range(len(products_collection)):
    
    
    # number of pages
    url = products_collection[n] + '?limit=96'
    browser.get(url)
    
    try:
        sleep(5)
        pages = browser.find_elements_by_xpath('//*[@class="pages"]//li')[-2].text
        pages = int(pages)
    except:
        pages=1
        pass
    
    print('\nSection:', n+1, '|', '#Pages:', pages,  '|', products_collection[n])
    
    section_links = []
    for page in range(1, pages+1):
        
        url = products_collection[n] + '?limit=96' + '&p={}'.format(page)
        browser.get(url)
        sleep(3)
        
        elems = browser.find_elements_by_xpath('//*[@class="products-list__action"]//a[@href]')
        page_links = []
        for elem in elems:
            link = elem.get_attribute("href")
            page_links.append(link)
        
        print('#', page, 'Got links for page:', len(page_links))
        section_links.extend(page_links)
    
    all_links.extend(section_links)
    print('Got links for section:', len(section_links))
   

In [ ]:
len(all_links)

In [ ]:
# Deduplicating links
product_links = []
for i in all_links:
    if i not in product_links:
        product_links.append(i)
len(product_links)

# Getting url options

In [ ]:
%%time
option_one_links = []
option_two_links = []
other_option_links = []
waiting_time = 15
for i in range(len(product_links))
    link = product_links[i]
    browser.get(link)
    sleep(2)
    
    # Dim One
    try:
        path = '//*[@id="product-options-wrapper"]/dl/dd[1]'
        first_input = browser.find_element_by_xpath(path).text.split('\n')
    except:
        pass
    
    for f_i, f_j in enumerate(first_input, start=1):
        path = '//*[@id="product-options-wrapper"]/dl/dd[1]/div/div/div[{}]'.format(f_i)
        try:
            element = WebDriverWait(browser, waiting_time).until(EC.element_to_be_clickable((By.XPATH, path)))
            element.click()
            break
        except:
            pass
    
    # Dim Two
    try:
        sleep(2)  
        path = '//*[@id="product-options-wrapper"]/dl/dd[2]'
        second_input = browser.find_element_by_xpath(path).text.split('\n')
    except:
        pass
    
    for s_i, s_j in enumerate(second_input, start=1):
        path = '//*[@id="product-options-wrapper"]/dl/dd[2]/div/div/div[{}]'.format(s_i)        
        sleep(1)
        
        try:
            element = WebDriverWait(browser, waiting_time).until(EC.element_to_be_clickable((By.XPATH, path)))
            element.click()
            break
        except:
            pass
        
    try:
       
        path = '//*[@id="product-options-wrapper"]/dl/dd[3]/div/div/div[1]'
        element = WebDriverWait(browser, waiting_time).until(EC.element_to_be_clickable((By.XPATH, path)))
        element.click()
        sleep(2)
        opt_three = browser.find_element_by_xpath(path).text
        if opt_one and opt_two and opt_three:
            print(i, 'Option One', link)
            option_one_links.append(link)
            continue
    except:
        pass

    # Check Option Two
    try:
       
        length_path = '//*[@id="product-options-wrapper"]/div[2]/dl/dd/div/div/input'
        browser.find_element_by_xpath(length_path).send_keys('-9999')
        print(i, 'Option Two', link)
        option_two_links.append(link)
    
    except:
        print(i, 'OTHER Option', link)
        other_option_links.append(link)
        pass

In [ ]:
# Check if all links were mapped to options
[x for x in product_links if x not in other_option_links and x not in option_two_links and x not in option_one_links]

In [ ]:
len(option_one_links), len(option_two_links)

# Option 1

In [ ]:
browser = webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver')

In [ ]:
main_df = pd.DataFrame()

In [ ]:
%%time
waiting_time = 20
start=1
for idx, page_link in enumerate(option_one_links[:], start=start):
    print('\nPage:', idx, 'out of', len(option_one_links))
    print('-'*80)
    browser.get(page_link)
    sleep(2)
    full_product_df = pd.DataFrame()
    
    # First Option
    path = '//*[@id="product-options-wrapper"]/dl/dd[1]/div'
    first_input = browser.find_element_by_xpath(path).text.split('\n')

    path = '//*[@id="product-options-wrapper"]/dl/dt[1]'
    first_option_name = browser.find_element_by_xpath(path).text
    opt_num = []
    for f_i, first_opt in enumerate(first_input, start=1):
        path = '//*[@id="product-options-wrapper"]/dl/dd[1]/div/div/div[{}]'.format(f_i)

        try:
            element = WebDriverWait(browser, waiting_time).until(EC.element_to_be_clickable((By.XPATH, path)))
            element.click()
        except:
            continue

        # Second Option
        sleep(4)  
        path = '//*[@id="product-options-wrapper"]/dl/dd[2]/div'
        second_input = browser.find_element_by_xpath(path).text.split('\n')

        path = '//*[@id="product-options-wrapper"]/dl/dt[2]'
        second_option_name = browser.find_element_by_xpath(path).text
        
        

        for s_i, second_opt in enumerate(second_input, start=1):

            sleep(4) 
            try:
                path = '//*[@id="product-options-wrapper"]/dl/dd[2]/div/div/div[{}]'.format(s_i)  
                element = WebDriverWait(browser, waiting_time).until(EC.element_to_be_clickable((By.XPATH, path)))
                element.click()
                sleep(2)
            except:
                continue

            # Initiating page dataset
            page_df = pd.DataFrame()

            # Third Option
            path = '//*[@id="product-options-wrapper"]/dl/dd[3]/div'
            third_input = browser.find_element_by_xpath(path).text.split('\n')

            path = '//*[@id="product-options-wrapper"]/dl/dt[3]'
            third_option_name = browser.find_element_by_xpath(path).text
            
            
            opt_num.append(len(third_input))
            
            for t_i, third_opt in enumerate(third_input, start=1): 

                # Specific product dataset
                product_df = pd.DataFrame([], index=[0])
                product_df[first_option_name + '-T'] = first_opt
                product_df[second_option_name + '-T'] = second_opt
                product_df[third_option_name + '-T'] = np.nan
                product_df['Price'] = np.nan
                product_df['Delivery'] = np.nan
                product_df['Description'] = np.nan

                
                sleep(1)

                try:
                    path = '//*[@id="product-options-wrapper"]/dl/dd[3]/div/div/div[{}]'.format(t_i)
                    element = WebDriverWait(browser, waiting_time).until(EC.element_to_be_clickable((By.XPATH, path)))
                    element.click()
                    sleep(1)

                    # Mapping Third Dimention
                    product_df[third_option_name + '-T'] = third_opt

                    # Price
                    sleep(1)
                    path = '//*[@class="price-box"]'
                    price = browser.find_element_by_xpath(path).text
                    product_df['Price'] = price

                    # Delivery
                    path = '//*[@class="product-view__delivery-time"]'
                    delivery = browser.find_element_by_xpath(path).text
                    product_df['Delivery'] = delivery

                    # Description
                    path = '//*[@class="product-view__collateral-main"]'
                    description = browser.find_element_by_xpath(path).text
                    product_df['Description'] = description

                    # Specification
                    html = browser.page_source
                    soup = BeautifulSoup(html, 'html.parser')
                    tech_data = {}
                    labels = soup.findAll("dt", { "class" : "technical-data__label" })
                    values = soup.findAll("dd", { "class" : "technical-data__value" })
                    for i in range(len(labels)):
                        tech_data[labels[i].text] = values[i].text
                    tech_data = pd.DataFrame([tech_data], index=[0])

                    product_df = pd.concat([product_df, tech_data], axis=1)
                    page_df = pd.concat([page_df, product_df], ignore_index=True)
                    page_df['URL'] = page_link
                    
                except:
                    pass
            try:  
                print(('Page Table Size:'), page_df.dropna(subset= [third_option_name + '-T']).shape[0], 'Expected Size', len(third_input))
            except:
                print(('Page Table Size:'), page_df.dropna().shape[0], 'Expected Size', len(third_input))
                pass
            
            full_product_df = pd.concat([full_product_df, page_df], ignore_index=True)
            
    est_qnt = np.sum(opt_num) 
    full_product_df['Est Quantity'] = est_qnt
    full_product_df['Actual Quantity'] = full_product_df.shape[0]
    
    try: 
        # Title
        path = '//*[@class="product-view__name"]'
        title = browser.find_element_by_xpath(path).text
        full_product_df['Title'] = title
    except:
        pass
        
    print('Estimated Quantity:', est_qnt, 'Actual Quantity:', full_product_df.shape[0])
    main_df = pd.concat([main_df, full_product_df], ignore_index=True)
    
    print('Main table size:', main_df.shape[0])
    sleep(2)
    
    if idx%10==0:
        with open('SCS-Wood-MainDataFrame-{}.pkl'.format(date.today().strftime("%d-%m-%Y")), 'wb') as f:
            pickle.dump(main_df, f)

# Option 2

In [ ]:
browser = webdriver.Chrome('/Users/macbook/Downloads/Chrome Driver/chromedriver')

In [ ]:
%%time
waiting_time = 25
second_main_df = pd.DataFrame()
for idx, page_link in enumerate(option_two_links[2:], start=3):
    print('\nPage:', idx)
    browser.get(page_link)
    full_product_df = pd.DataFrame()
    
    dead_buttons['Link-{}'.format(idx)] = {}
    dead_buttons['Link-{}'.format(idx)]['Option 1'] = []
    
    # First Option
    sleep(3)
    path = '//*[@id="product-options-wrapper"]/dl/dd[1]/div'
    first_input = browser.find_element_by_xpath(path).text.split('\n')

    path = '//*[@id="product-options-wrapper"]/dl/dt[1]'
    first_option_name = browser.find_element_by_xpath(path).text
    opt_num = []
    for f_i, first_opt in enumerate(first_input, start=1):
        dead_buttons['Link-{}'.format(idx)]['Option 1-{}'.format(f_i)] = []
        
        path = '//*[@id="product-options-wrapper"]/dl/dd[1]/div/div/div[{}]'.format(f_i)
        print(' \nFirst option:', f_i)
        try:
            element = WebDriverWait(browser, waiting_time).until(EC.element_to_be_clickable((By.XPATH, path)))
            element.click()
        except:
            dead_buttons['Link-{}'.format(idx)]['Option 1'].append(f_i)
            print(dead_buttons)
            continue

        # Second Option
        sleep(2)  
        path = '//*[@id="product-options-wrapper"]/dl/dd[2]/div'
        second_input = browser.find_element_by_xpath(path).text.split('\n')

        path = '//*[@id="product-options-wrapper"]/dl/dt[2]'
        second_option_name = browser.find_element_by_xpath(path).text
        opt_num.append(len(second_input))
        
        for s_i, second_opt in enumerate(second_input, start=1):
            print('Second option:', s_i)

            sleep(6) 
            try:
                path = '//*[@id="product-options-wrapper"]/dl/dd[2]/div/div/div[{}]'.format(s_i)  
                element = WebDriverWait(browser, waiting_time).until(EC.element_to_be_clickable((By.XPATH, path)))
                element.click()
                sleep(5)
                
                # Delivery
                path = '//*[@class="product-view__delivery-time"]'
                delivery = browser.find_element_by_xpath(path).text

                # Description
                path = '//*[@class="product-view__collateral-main"]'
                description = browser.find_element_by_xpath(path).text

                # Specification
                html = browser.page_source
                soup = BeautifulSoup(html, 'html.parser')
                tech_data = {}
                labels = soup.findAll("dt", { "class" : "technical-data__label" })
                values = soup.findAll("dd", { "class" : "technical-data__value" })
                for i in range(len(labels)):
                    tech_data[labels[i].text] = values[i].text
                tech_data = pd.DataFrame([tech_data], index=[0])

            except:
                dead_buttons['Link-{}'.format(idx)]['Option 1-{}'.format(f_i)].append(s_i)
                print(dead_buttons)
                continue

            # Initiating page dataset
            page_df = pd.DataFrame()

            # Third Option
            # Max Lenght
            length_path = '//*[@id="product-options-wrapper"]/div[2]/dl/dd/div/div/input'
            browser.find_element_by_xpath(length_path).send_keys('20000')
            browser.find_element_by_xpath(length_path).send_keys(Keys.ENTER)

            path = '/html/body/div[2]/div/div[4]/div/div[2]/div[2]/form/div/div[2]/div[5]/div[2]/dl/dt/label/div'
            max_length = browser.find_element_by_xpath(path).text.split('=')[0]
            max_length = re.sub(' ','', max_length)
            sleep(2)
            browser.find_element_by_xpath(length_path).clear()

            # Min Lenght
            browser.find_element_by_xpath(length_path).send_keys('100')
            browser.find_element_by_xpath(length_path).send_keys(Keys.ENTER)
            path = '/html/body/div[2]/div/div[4]/div/div[2]/div[2]/form/div/div[2]/div[5]/div[2]/dl/dt/label/div'
            min_length = browser.find_element_by_xpath(path).text.split('=')[0]
            min_length = re.sub(' ', '', min_length)
            sleep(2)
            browser.find_element_by_xpath(length_path).clear()
            
            third_option_name = 'Länge'

            for third_opt in range(int(min_length), int(max_length)+step, step):
                browser.find_element_by_xpath(length_path).send_keys(third_opt)
                
                # Specific product dataset
                product_df = pd.DataFrame([], index=[0])
                product_df[first_option_name + '-T'] = first_opt
                product_df[second_option_name + '-T'] = second_opt
                product_df[third_option_name + '-T'] = third_opt
                product_df['Delivery'] = delivery
                product_df['Description'] = description
                product_df['Price'] = np.nan

                try:
                    sleep(0.2)
                    # Price
                    path = '//*[@class="price-box"]'
                    price = browser.find_element_by_xpath(path).text
                    product_df['Price'] = price

                    product_df = pd.concat([product_df, tech_data], axis=1)
                    page_df = pd.concat([page_df, product_df], ignore_index=True)
                    page_df['URL'] = page_link
                    
                except:
                    pass
                
                sleep(0.1)
                browser.find_element_by_xpath(length_path).clear()
            
            full_product_df = pd.concat([full_product_df, page_df], ignore_index=True)
            print('Page Product Table:', page_df.shape[0])
        
        print('Full Product Table:', full_product_df.shape[0])
    try: 
        # Title
        path = '//*[@class="product-view__name"]'
        title = browser.find_element_by_xpath(path).text
        full_product_df['Title'] = title
    except:
        pass
    
    est_qnt = np.sum(opt_num)*len(range(int(min_length), int(max_length)+step, step))
    full_product_df['Est Quantity'] = est_qnt
    full_product_df['Actual Quantity'] = full_product_df.shape[0]
        
    print('Estimated Quantity:', est_qnt, 'Actual Quantity:', full_product_df.shape[0])
    second_main_df = pd.concat([second_main_df, full_product_df], ignore_index=True)
    
    print('Main table size:', second_main_df.shape[0])
    sleep(3)
    with open('SCS-Wood-SecondMainDataFrame-{}.pkl'.format(date.today().strftime("%d-%m-%Y")), 'wb') as f:
        pickle.dump(second_main_df, f)

### Images

In [ ]:
data = pd.concat([main_df, second_main_df]

In [ ]:
images_df = pd.DataFrame({'Images':np.nan, 'URL':data['URL'].unique()}, index=range(data['URL'].nunique()))
images_df.shape

In [ ]:
%%time
for idx in images_df.index:
    
    try:
        url = images_df['URL'].loc[idx]
        browser.get(url)
        sleep(1.5)
    
        # Click to expand the image
        path = '//*[@class="product-view__main-image product-image product-image-zoom"]'
        browser.find_elements_by_xpath(path)[0].click()
        sleep(1)

        # Get the image
        path = '//*[@class="mz-figure mz-hover-zoom mz-ready"]/img'
        img = browser.find_elements_by_xpath(path)[0].get_attribute('src')
        images_df['Images'].loc[idx]= img
        
        print('#',idx, url ,img)
      
    except:
        print('ERROR WITH IMAGE')
        continue

In [ ]:
images_df['Images'] = images_df['Images'].apply(lambda x: np.nan if x=='' else x)
data = pd.merge(data, images_df, on='URL', how='left')
data.head(2)